In [10]:
from IPython.display import display, HTML

display(
    HTML(
        """<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""
    )
)

[ RAG 구현 절차 ]

1. 문서의 내용을 읽는다(document_loader를 이용)

- (1) https://python.langchain.com/v0.2/docs/integrations/document_loaders/
- (2) https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
- %pip install --upgrade --quiet docx2txt

2. 문서를 쪼갠다(한번에 이해하고 처리할 수 있는 입력+출력 토큰수가 제한)

- (1) https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries
- %pip install -qU langchain-text-splitters

3. 쪼갠 문서를 임베딩하여 vector database에 넣음

- (1) OpenAIEmbeddings나 UpstageEmbeddings이용해서 임베딩
- (2) https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/
- %pip install –q langchain-chroma

4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문으로 전달하여 답변 얻음(제공되는 Prompt활용)

- (1) https://python.langchain.com/v0.2/docs/tutorials/rag/
- %pip install –q langchain langchainhub

- https://smith.langchain.com/ 에서 key 생성 .env key 추가


# 2. 문서를 쪼개면서 읽기 (O)


In [11]:
import time

start = time.time()

from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
)  # 문서를 읽어오면서 겹치게 발라줌

loader = Docx2txtLoader("./tax_docs/소득세법(법률)(제20615호)(20250701).docx")

# 문자단위로 쪼갬
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=200  # 문서를 쪼갤떄 1500글자씩 쪼개
)
# 1번째 chunk 1 ~ 1450글자
# 2번쨰 chunk 1250 ~ 1750글자

documents = loader.load_and_split(text_splitter=text_splitter)

runtime = time.time() - start
print("문서 쪼개면서 읽는 시간", runtime)

문서 쪼개면서 읽는 시간 1.8499813079833984


# 3. 쪼갠문서를 임베딩 -> 벡터 데이터베이스 저장

- Embedding Model : Upstage solar-embedding-1-large, embedding-query
- 벡터 데이터베이스 : chroma


In [12]:
# 사용법 : https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage/
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()
embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large",
    # model="embedding-query"
)

In [13]:
doc_result = embeddings.embed_documents(
    ["소득세법 어쩌구 저쩌구", documents[0].page_content]
)
print(len(doc_result), len(doc_result[0]), len(doc_result[1]))

2 4096 4096


In [14]:
%%time
from langchain_chroma import Chroma

# 데이터를 처음 저잘할떄
# database = Chroma.from_documents(
#     documents=documents,
#     embedding=embeddings,
#     collection_name='tax_collection',# 생략시 이름 랜덤
#     persist_directory='./chroma_upstage',# 생략시 로컬데이터베이스에 저장안됨, 프로그램 종료시 db날라감
# )
# 계속 실행하면 하면 안됨. 꼭 한번만. 그래서 주석처리


CPU times: user 5 μs, sys: 0 ns, total: 5 μs
Wall time: 5.72 μs


In [15]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma
%time
load_dotenv()
embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large",
    # model="embedding-query"
)

# 이미 저장된 vector DB를 사용할떄
database = Chroma(
    embedding_function=embeddings,
    collection_name='tax_collection',
    persist_directory='./chroma_upstage'
)

CPU times: user 1 μs, sys: 1 μs, total: 2 μs
Wall time: 2.15 μs


# 4. vector DB 에 질문과 유사도 검색 (답변 생성을 위한 retrieval)


In [16]:
query = "연봉 5000만원인 직장인의 소득세는 얼마인가요?"

retrieved_docs = database.similarity_search(query, k=3)  # 기본 k값이 4

In [17]:
retrieved_docs

[]

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM 전달하여 답변 생성


In [18]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

In [19]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
[context]는 다음과 같습니다.
{retrieved_docs}
Question: {query}
"""

In [20]:
ai_message = llm.invoke(prompt)

In [21]:
print(ai_message.content)

연봉이 5,000만 원인 직장인의 소득세 계산은 여러 공제와 세율에 따라 달라질 수 있지만, 기본적인 계산 방법을 안내드리겠습니다.

1. **근로소득세 과세표준 산정:**
- 연봉 5,000만 원에서 근로소득공제와 기타 공제(인적공제, 보험료공제 등)를 차감한 금액이 과세표준이 됩니다.

2. **근로소득공제 계산:**
- 1,200만 원 이하: 80%
- 1,200만 원 초과 4,600만 원 이하: 1,200만 원 + (연봉 - 1,200만 원) × 40%
- 4,600만 원 초과: 연봉 × 15% + 1,560만 원

연봉 5,000만 원의 경우,
근로소득공제는:
4,600만 원 이하 구간이 아니므로, 4,600만 원 초과 구간에 해당합니다.
공제액 = 1,200만 원 + (5,000만 원 - 1,200만 원) × 40%
= 1,200만 원 + 3,800만 원 × 40%
= 1,200만 원 + 1,520만 원
= 2,720만 원

이에 따라,
과세표준 ≈ 연봉 5,000만 원 - 공제액 2,720만 원 = 2,280만 원

3. **기본 세율 적용:**
연간 소득세 표에 따라, 과세표준 2,280만 원은 다음 구간에 해당:
- 1,200만 원 초과 ~ 4,600만 원 이하: 15%

초과 구간에 따라 세율과 누진세액이 적용됩니다.

4. **세액 계산 (대략):**
- 1,200만 원까지: 1,200만 원 × 6% = 72,000원
- 나머지 1,080만 원: 1,080만 원 × 15% = 162만 원
- 합산: 약 162만 원 + 7만2천 원 = 234만 원

여기에 주민세(10%)와 기타 공제, 세액공제 등을 반영하지 않은 단순 계산이며, 실제 세액은 인적공제, 보험료공제, 연금보험료공제, 소득세 세액공제 등에 따라 차이가 있을 수 있습니다.

**요약:**  
대략적인 소득세는 약 230만 원 내외로 예상되며, 상세 계산과 정확한 세액 산정을 위해서는 맞춤 공제 항목과 세액공제를 반영하는 것이 필요합니다.

추가 문의 사항이나 구체적인 공제 내용이 

# 5. Agumentation 을 위한 제공하는 Prompt 활용하여, langchain으로 답변 생성

- 사용법 : https://python.langchain.com/v0.2/docs/tutorials/rag/


In [22]:
query = "연봉 5000만원인 직장인의 소득세는 얼마인가요?"

from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

## RetrievalQA를 통해 LLM전달 (create_retrieval_chain이 대체)

```
 query -> retriever 전달(벡터 검색 수행)
 -> retriever문서 -> prompt의 {context}에 삽입
 -> query -> prompt의 (question)에 삽입
```


In [23]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(search_kwargs={"k": 5}),
    chain_type_kwargs={"prompt": prompt},
)

In [24]:
ai_message = qa_chain.invoke({"query": query})

In [25]:
ai_message

{'query': '연봉 5000만원인 직장인의 소득세는 얼마인가요?',
 'result': '소득세는 과세표준, 세율, 공제액 등에 따라 달라집니다. 연봉 5000만원의 경우, 총소득세 계산은 복잡하며, 공제액과 세율에 따라 달라집니다. 정확한 금액을 알기 위해서는 상세한 세법 규정을 참고하거나 세무 전문가와 상담하는 것이 좋습니다.'}